In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats 

In [3]:
gett = pd.read_csv('Netology_A_B-testing_HW_1.csv')
gett.head()

,Unnamed: 0,date,user_id,hour,app_opened,price_seen,order_made,surge,ride_completed,user_cancelled,age,os,city_center_order,order_class,distance,rfm
0,0,2020-05-21,867689,12,1,1,1,no surge,1,0,20,iOS,0,business,7.982135,low
1,1,2020-05-23,752172,5,1,1,1,no surge,1,0,37,Android,1,economy,2.908468,high
2,2,2020-05-20,486559,15,1,1,1,no surge,1,0,47,Android,0,comfort,7.224614,low
3,3,2020-05-19,304024,0,1,1,1,no surge,1,0,59,Android,1,economy,1.874349,low
4,4,2020-05-23,139420,0,1,1,1,no surge,1,0,19,Android,0,business,10.704778,medium


### Особенность - "Для устройств Android наблюдаем более высокие юзерские отмены"

In [41]:
gett_and = gett[gett['os'] == 'Android']['user_cancelled'].sample(n = 44000)
gett_and.value_counts()

0    38277
1     5723
Name: user_cancelled, dtype: int64

In [42]:
gett_ios = gett[gett['os'] == 'iOS']['user_cancelled'].sample(n = 44000)
gett_ios.value_counts()

0    40156
1     3844
Name: user_cancelled, dtype: int64

In [43]:
T, p = stats.ttest_ind(gett_and, gett_ios) 
T, p

(20.396235374559524, 2.9551889999638343e-92)

In [33]:
# пи-вэлью крайне мал, следовательно вероятность ошибиться, отвергнув Н0 (что различий нет), практически отсутствует. 
# Поэтому различия в выборках действительно есть! Т.е. у андроидов отмены встречаются чаще.


### Особенность - "В период Surge видим более низкую price-to-order конверсию"

In [40]:
gett_sur = gett[(gett['surge'] == 'surge') & gett['price_seen'] == 1]['order_made'].sample(n = 25000)
gett_sur.value_counts()

1    18060
0     6940
Name: order_made, dtype: int64

In [39]:
gett_nosur = gett[(gett['surge'] == 'no surge') & gett['price_seen'] == 1]['order_made'].sample(n = 25000)
gett_nosur.value_counts()

1    21442
0     3558
Name: order_made, dtype: int64

In [44]:
T2, p2 = stats.ttest_ind(gett_sur, gett_nosur) 
T2, p2

(-37.658285062839425, 4.695725650156552e-306)

In [45]:
# пи-вэлью крайне мал, следовательно вероятность ошибиться, отвергнув Н0 (что различий нет), практически отсутствует. 
# Поэтому различия в выборках действительно есть! Т.е. с наценкой конверсия в заказ менее вероятна, чем без неё.